In [1]:
import numpy as np
import matplotlib
matplotlib.use('QtAgg') 
from matplotlib.animation import PillowWriter
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.integrate import RK45


### Equations:
[1] https://www.myphysicslab.com/pendulum/double-pendulum-en.html \
[2] https://www.jousefmurad.com/engineering/double-pendulum-1/ 

In [8]:
# Stałe fizyczne
przyspieszenie_ziemskie = 9.81  # Przyspieszenie ziemskie
dlugosc_wahadla_1 = 1.25         # Długość pierwszego wahadła
dlugosc_wahadla_2 = 2.0         # Długość drugiego wahadła
masa_wahadla_1 = 1.5            # Masa pierwszego wahadła
masa_wahadla_2 = 2.25           # Masa drugiego wahadła

# Parametry symulacji
kat_poczatkowy_1 = np.pi / 2    # Kąt początkowy pierwszego wahadła
kat_poczatkowy_2 = np.pi        # Kąt początkowy drugiego wahadła
krok_czasowy = 0.005            # Krok czasowy
czas_symulacji = 60             # Czas symulacji

# Funkcja lambda do obliczania Hamiltonianu
oblicz_hamiltonian = lambda theta1, theta2, w1, w2: (
    0.5 * masa_wahadla_1 * (dlugosc_wahadla_1 * w1) ** 2
    + 0.5
    * masa_wahadla_2
    * (
        (dlugosc_wahadla_1 * w1) ** 2
        + (dlugosc_wahadla_2 * w2) ** 2
        + 2
        * dlugosc_wahadla_1
        * dlugosc_wahadla_2
        * w1
        * w2
        * np.cos(theta1 - theta2)
    ),
    -masa_wahadla_1 * przyspieszenie_ziemskie * dlugosc_wahadla_1 * np.cos(theta1)
    - masa_wahadla_2
    * przyspieszenie_ziemskie
    * (dlugosc_wahadla_1 * np.cos(theta1) + dlugosc_wahadla_2 * np.cos(theta2)),
)

def pochodne(y):
    """
    [1] Równania różniczkowe dla wahadła podwójnego zostały wyprowadzone
    przy użyciu równań Lagrange'a drugiego rodzaju. 
    """
    theta1, theta2, w1, w2 = y

    mianownik = (
        2 * masa_wahadla_1
        + masa_wahadla_2
        - masa_wahadla_2 * np.cos(2 * theta1 - 2 * theta2)
    )

    dtheta1 = w1
    dtheta2 = w2
    dw1 = (
        -przyspieszenie_ziemskie * (2 * masa_wahadla_1 + masa_wahadla_2) * np.sin(theta1)
        - masa_wahadla_2 * przyspieszenie_ziemskie * np.sin(theta1 - 2 * theta2)
        - 2
        * np.sin(theta1 - theta2)
        * masa_wahadla_2
        * (w2 * w2 * dlugosc_wahadla_2 + w1 * w1 * dlugosc_wahadla_1 * np.cos(theta1 - theta2))
    ) / (dlugosc_wahadla_1 * mianownik)
    dw2 = (
        2
        * np.sin(theta1 - theta2)
        * (
            w1 * w1 * dlugosc_wahadla_1 * (masa_wahadla_1 + masa_wahadla_2)
            + przyspieszenie_ziemskie * (masa_wahadla_1 + masa_wahadla_2) * np.cos(theta1)
            + w2 * w2 * dlugosc_wahadla_2 * masa_wahadla_2 * np.cos(theta1 - theta2)
        )
    ) / (dlugosc_wahadla_2 * mianownik)

    # Obliczenie energii całkowitej, kinetycznej i potencjalnej układu
    Kin, Vpot = oblicz_hamiltonian(theta1, theta2, w1, w2)
    Etot = Kin + Vpot
    return [dtheta1, dtheta2, dw1, dw2, Etot, Kin, Vpot]

# Funkcja lambda do obliczania pochodnych
pochodne_rk45 = lambda t, y: pochodne(y)[:4]


def oblicz_energie(wyniki):
    """Oblicza energie dla każdego kroku czasowego w symulacji."""
    energie_calkowite = []
    energie_kinetyczne = []
    energie_potencjalne = []
    for wynik in wyniki:
        theta1, theta2, w1, w2 = wynik
        Kin, Vpot = oblicz_hamiltonian(theta1, theta2, w1, w2)
        Etot = Kin + Vpot
        energie_calkowite.append(Etot)
        energie_kinetyczne.append(Kin)
        energie_potencjalne.append(Vpot)
    return (
        np.array(energie_calkowite),
        np.array(energie_kinetyczne),
        np.array(energie_potencjalne),
    )
    
def symulacja(
    kat_poczatkowy_1,
    kat_poczatkowy_2,
    krok_czasowy,
    czas_symulacji,
):
    """Symulacja wahadła dwuciałowego."""

    # Warunki początkowe
    warunki_poczatkowe = np.array([kat_poczatkowy_1, kat_poczatkowy_2, 0.0, 0.0])

    # Rozwiązanie układu równań różniczkowych
    rk45 = RK45(
        pochodne_rk45, 0, warunki_poczatkowe, czas_symulacji, max_step=krok_czasowy
    )
    czasy = []
    wyniki = []
    while rk45.status == "running":
        rk45.step()
        czasy.append(rk45.t)
        wyniki.append(rk45.y)
    wyniki = np.array(wyniki)

    # Obliczenie energii dla każdego kroku czasowego
    Etot, Kin, Vpot = oblicz_energie(wyniki)

    # Dodanie energii do tablicy wyniki
    wyniki = np.column_stack((wyniki, Etot, Kin, Vpot))

    return wyniki, Etot, Kin, Vpot, czasy

def animacja(wyniki, energie_calkowite, energie_kinetyczne, energie_potencjalne, czasy, krok_czasowy):
    """Tworzy animację wahadła."""

    theta1, theta2 = wyniki[:, 0], wyniki[:, 1]

    # Współrzędne x, y mas
    x1 = dlugosc_wahadla_1 * np.sin(theta1)
    y1 = -dlugosc_wahadla_1 * np.cos(theta1)
    x2 = x1 + dlugosc_wahadla_2 * np.sin(theta2)
    y2 = y1 - dlugosc_wahadla_2 * np.cos(theta2)

    # Ustawienia wykresu
    fig = plt.figure(figsize=(16, 8))
    plt.tight_layout()
    ax1 = fig.add_subplot(121, autoscale_on=False, xlim=(-np.max(dlugosc_wahadla_1+dlugosc_wahadla_2),
                                                         np.max(dlugosc_wahadla_1+dlugosc_wahadla_2)),
                                                   ylim=(-np.max(dlugosc_wahadla_1+dlugosc_wahadla_2),
                                                          np.max(dlugosc_wahadla_1+dlugosc_wahadla_2)))
    ax1.set_aspect("equal")
    ax1.grid()

    # Obliczenie minimalnej i maksymalnej wartości dla ylim
    ylim_min = np.min(
        [
            np.min(energie_calkowite),
            np.min(energie_kinetyczne),
            np.min(energie_potencjalne),
        ]
    )
    ylim_max = np.max(
        [
            np.max(energie_calkowite),
            np.max(energie_kinetyczne),
            np.max(energie_potencjalne),
        ]
    )

    ax2 = fig.add_subplot(
        122, xlim=(0, czasy[-1]), ylim=(ylim_min, ylim_max)
    )  
    ax2.set_xlabel("Czas (s)")
    ax2.set_ylabel("Energia (J)")
    ax2.grid()

    # Elementy animacji
    (linia,) = ax1.plot([], [], "o-", lw=2)
    szablon_czasu = "czas = %.1fs"
    tekst_czasu = ax1.text(0.05, 0.9, "", transform=ax1.transAxes)
    (linia_E,) = ax2.plot([], [], "k-", lw=2, label="E")
    (linia_T,) = ax2.plot([], [], "r-", lw=2, label="T")
    (linia_V,) = ax2.plot([], [], "b-", lw=2, label="V")
    ax2.legend()

    def init():
        linia.set_data([], [])
        tekst_czasu.set_text("")
        linia_E.set_data([], [])
        linia_T.set_data([], [])
        linia_V.set_data([], [])
        return linia, tekst_czasu, linia_E, linia_T, linia_V

    def animate(i):
        # Obliczenie współrzędnych x, y mas dla aktualnej klatki
        thisx = [0, dlugosc_wahadla_1 * np.sin(wyniki[i, 0]), 
                 dlugosc_wahadla_1 * np.sin(wyniki[i, 0]) + dlugosc_wahadla_2 * np.sin(wyniki[i, 1])]
        thisy = [0, -dlugosc_wahadla_1 * np.cos(wyniki[i, 0]), 
                 -dlugosc_wahadla_1 * np.cos(wyniki[i, 0]) - dlugosc_wahadla_2 * np.cos(wyniki[i, 1])]

        linia.set_data(thisx, thisy)
        tekst_czasu.set_text(szablon_czasu % (i * krok_czasowy))
        linia_E.set_data(np.array(czasy[:i]), energie_calkowite[:i])
        linia_T.set_data(np.array(czasy[:i]), energie_kinetyczne[:i])
        linia_V.set_data(np.array(czasy[:i]), energie_potencjalne[:i])
        return linia, tekst_czasu, linia_E, linia_T, linia_V

    ani = FuncAnimation(
        fig,
        animate,
        np.arange(1, len(wyniki)),
        interval=krok_czasowy * 1,
        blit=True,
        init_func=init,
    )

    # Wyświetlenie wykresu energii w czasie
    plt.figure()
    plt.plot(czasy, energie_calkowite, "k-", lw=2, label="E")
    plt.plot(czasy, energie_kinetyczne, "r-", lw=2, label="T")
    plt.plot(czasy, energie_potencjalne, "b-", lw=2, label="V")
    plt.xlabel("Czas (s)")
    plt.ylabel("Energia (J)")
    plt.title("Energia w czasie")
    plt.legend()
    plt.show()

    return ani

# Symulacja
(
    wyniki,
    energie_calkowite,
    energie_kinetyczne,
    energie_potencjalne,
    czasy,
) = symulacja(
    kat_poczatkowy_1,
    kat_poczatkowy_2,
    krok_czasowy,
    czas_symulacji,
)

# Animacja
animacja = animacja(
    wyniki,
    energie_calkowite,
    energie_kinetyczne,
    energie_potencjalne,
    czasy,
    krok_czasowy,
)
